In [7]:
from dotenv import load_dotenv
import os


Config the model

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')
output = model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


Config the embedding model


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

384

In [10]:
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain_community.vectorstores import chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
loader = DirectoryLoader("../data",glob="./*.txt",loader_cls=TextLoader)